# Cleaning and transforming data

Files needed = ('atussum_2017.dat', movie lens small data set)

We now know how to create some complicated datasets. In this notebook we will continue practicing techniques for manipulating data into forms that are amenable to analysis.  

We will also learn about the American Time Use Survey. 

In [ ]:
import pandas as pd                 # pandas for data handling
import matplotlib.pyplot as plt     # matplotlib for plots
import numpy as np                  # numpy for numerical methods 

## American Time Use Survey (ATUS)
The Bureau of Labor Statistics oversees the [American Time Use Survey](https://www.bls.gov/tus/), which asks a sample of Americans to complete detailed diaries keeping track of each minute of their day. 

Follow this link [https://www.bls.gov/tus/datafiles_2017.htm](https://www.bls.gov/tus/datafiles_2017.htm) to the page for the 2017 survey. Download the **ATUS 2017 Activity summary file (zip)** file located in the **2017 Basic ATUS Data Files** section of the page. Alternatively, download it directly [https://www.bls.gov/tus/special.requests/atussum_2017.zip](https://www.bls.gov/tus/special.requests/atussum_2017.zip). 

Unzip the file. We are looking for `atussum_2017.dat`. It is a coma separated file. Let's get it loaded.

In [ ]:
atus = pd.read_csv('atussum_2017/atussum_2017.dat')

In [ ]:
atus.head()

421 variables! That's too many for us today. Let's just keep a few. 

In [ ]:
col_list=atus.columns.tolist()   # columns returns a list object. Turn it into a list for easier printing.

# Print the first and last 20 variables. 
print(col_list[0:20],'\n')
print(col_list[-20:])

The demographic variables are all uppercase letters. The time variables are of the form 'txxyyzz' where xx is the major category code, yy is the second-tier code, and zz is the third-tier code. ([docs](https://www.bls.gov/tus/lexiconnoex2017.pdf))

Let's keep some demographic data and some data about working and sleeping. 

In [ ]:
vars_to_keep = ['TEAGE', 'TESEX', 'PTDTRACE', 'PEEDUCA', 'GTMETSTA', 'TELFS', 
                'TUDIARYDAY', 't050101', 't050102', 't050201', 't010101', 't010102']

atus_small = atus[vars_to_keep]

In [ ]:
atus_small.head()

Let's give the columns some more reasonable names. 

In [ ]:
# See https://www.bls.gov/tus/freqvariables.pdf for frequently used variables
atus_small = atus_small.rename(columns={'TEAGE':'age', 'TESEX':'gender', 'PTDTRACE':'race', 
                           'PEEDUCA':'edu', 'GTMETSTA':'metro', 'TELFS':'employ', 'TUDIARYDAY':'day'})
atus_small.head()

In [ ]:
# See https://www.bls.gov/tus/lexiconnoex2017.pdf for definitions
atus_small = atus_small.rename(columns={'t050101':'work_main', 't050102':'work_other', 't010101':'sleep', 
                                        't050201':'work_soc', 't010102':'no_sleep'})
atus_small.head()

Okay, we have variable names that we can live with. Let's start transforming the data to get it ready for analysis. 

### A word on loops
We have seen many places where a loop saved us time and effort. It might seem natural to use a loop to perform an operation on each element of a column or a DataFrame. 

In general, however, we want to avoid this. Instead, we have used pandas vectorized operations such as 
```python
x['new_var'] = x['var_1'] / x['var_2']
```
to perform element-wise division. Using pandas' native operations is much (much much) faster than looping over the rows of a DataFrame. A lot of optimization has been written into the native functions that is not there when we loop ourselves. 

Fortunately, pandas provides methods that let us to very complex and very general operations to a DataFrame without resorting to a loop. We consider a few of these below. 

### replace( ) 

The gender variable is coded 1 for male and 2 for female. I do not want to have to remember that!

The `replace( )` method replaces one value for another. One syntax is 
```
atus_small['gender'] = atus_small['gender'].replace(1, 'male')
```
but a more powerful one passes a dict or a list.
```
atus_small['gender'] = atus_small['gender'].replace({1:'male', 2:'female'})
```

In [ ]:
gender_codes = {1:'male', 2:'female'}
atus_small['gender'] = atus_small['gender'].replace(gender_codes)
atus_small.head()

### unique( )
Let's code race, too. What codes are in our data? The method `unique( )` returns the unique values.

In [ ]:
races = atus_small['race'].unique()
races.sort()
print(races)


In [ ]:
# I looked up the codes in the documentation...

race_codes = {1:'white', 2:'black', 3:'native_am', 4:'asian', 5:'hawaiian', 6:'wh_bl', 
              7:'wh_na', 8:'wh_as', 9:'wh_ha', 10:'bl_na', 11:'bl_as', 13:'na_as', 16:'wh_bl_as', 18:'wh_as_ha'}

In [ ]:
atus_small['race'] = atus_small['race'].replace(race_codes)

In [ ]:
atus_small.head()

### Apply a function to a column: map( )

We can apply functions to the individual elements in a column, too. The `map()` functions handles this for us. It applies the given function to each element of the column. These can be built-in functions, or user-defined functions. This quite powerful. We can write a function that performs a complicated transformation and apply to each element of a column in one simple line.  

Let's define a function that converts minutes to hours. 

In [ ]:
def minutes_to_hours(x):
    return x/60

Now apply `map( )` to the work column. 

\[This example is a bit contrived. We could have just as easily done `atus_small['work_main']/60`. But that wouldn't have given us practice with `map( )`.\] 

In [ ]:
atus_small['work_main'] = atus_small['work_main'].map(minutes_to_hours)
atus_small.head()

### Apply a function to a DataFrame: applymap( )
If we want to apply the same function to each element in several columns of a DataFrame (rather than a Series) we use `applymap( )`. It works the same way, applying the function to each element. 

\[I am not sure why the developers of pandas created separate method for a Series vs a DataFrame. My best guess is that each method takes advantage of Series- or Dataframe-specific functionality for better performance.\]

In [ ]:
# We can map to several columns at once.
atus_small[['work_other', 'sleep', 'no_sleep']] = atus_small[['work_other', 'sleep', 'no_sleep']].applymap(minutes_to_hours)
atus_small.head()

## Practice
Take a few minutes and try the following. Feel free to chat with those around you if you get stuck. 

1. `educ` holds the highest level of education obtained. It can take values between 31 and 46. Covert the numeric values to labels. 

   * codes < 39 are 'less than high'
   * 39 = 'high school'
   * 40 = 'some college'
   * 41 & 42 are 'associate'
   * 43 = 'bachelor'
   * 44 = 'master'
   * 45 = 'prof'
   * 46 = 'phd'
   
   
This might be useful: 
```python
educ_codes = {39:'high school', 40:'some college', 41:'associate', 42:'associate', 
              43: 'bachelor', 44: 'master', 45:'prof', 46:'phd'}
```

2. Make sure you covered all your bases. Print out a list of the unique values in the column 'edu'.

3. Apply the `minutes_to_hours` function to the `work_soc` variable. This variable is the time spent 'Socializing, relaxing, and leisure as part of job'. I have no idea what kind of jobs make you relax and take leisure. 

4. Create a column named 'work' that is the sum of 'work_main' and 'work_other'.

5. Create a histogram of 'work' for only those observations with $\text{work}>0$. 

### Binning: cut( )

Often we would like to take a continuous variable and make it discrete. One way to do this is to place the data into 'bins' or 'buckets'. (This is implicitly being done when we create a histogram.) Pandas has a nice facility for this. 

In [ ]:
# What range are the ages?
atus_small['age'].describe()

In [ ]:
# 15 to 85 years 

bins = [15, 25, 35, 45, 55, 65, 75]  # These are the endpoints for the bins

atus_small['age_bin'] = pd.cut(atus_small['age'], bins)


In [ ]:
atus_small.head(10)

I appreciate the bin notation: The open and closed ends of the interval are clearly indicated. 

Notice the NaN for observation 2. Our lowest bin starts at 15, but it is open on the left end point --- 15 is not included in the bin. We could set the bottom interval to 14 to catch this observation. We have a similar problem with observation 4.

In [ ]:
bins = [14, 25, 35, 45, 55, 65, 75]  # These are the endpoints for the bins
atus_small['age_bin'] = pd.cut(atus_small['age'], bins)
atus_small.head(10)

We could also make the bins closed on the left and open on the right by passing `'right=False'` parameter to `cut( )`. 

In [ ]:
bins = [15, 25, 35, 45, 55, 65, 75]  # These are the endpoints for the bins
atus_small['age_bin'] = pd.cut(atus_small['age'], bins, right=False)
atus_small.head(10)

We are still missing observation 4.

`.cut( )` can create bins for us, too. Pass cut an integer and get back that many equally-sized bins. The `precision` parameter determines how many decimal places are used in the bin edges. 

In [ ]:
# Tell .cut to make three bins. It will choose the end points. 
atus_small['age_bin'] = pd.cut(atus_small['age'], bins = 3, precision = 0)
atus_small.head(10)

In [ ]:
pd.value_counts(atus_small['age_bin'])  # How many observations in each bin?

### Quantiles: qcut( )
Notice that the bins are equally spaced from the smallest to the largest values of age. The number of observations in each is not equal. 

The method `.qcut( )` splits the data according to the sample quantiles. This should get you bins of approximately equal numbers of observations. 

In [ ]:
# Create quartiles (4 bins) based on age
atus_small['age_bin'] = pd.qcut(atus_small['age'], q = 4)  
atus_small.head(10)

In [ ]:
pd.value_counts(atus_small['age_bin'])  # How many observations in each bin?

### String methods
We have seen some of these before. When we learned about strings. These are analogous to the string methods in standard python, but they have been optimized for DataFrames. These *vectorized string methods*, work element-wise over an entire column. The method call looks like

```python
data['var'].str.method()
```

where `.method( )` is the method we are applying. A list of vectorized string methods is on page 218 in McKinney and in the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#method-summary). Below, we try a few out. 

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

### str.contains( )
The genres are mixed together. Let's get all the comedies. The `.contains( )` method returns a bool Series with True for observations in which the string contains the search term.

In [ ]:
movies['genres'].str.contains('Comedy')

In [ ]:
comedies = movies[movies['genres'].str.contains('Comedy')]
comedies.head()

### str.split( )
This method splits the string up at the delimiter that is passed to split( ). It returns a list of each chunk that falls between the delimiter. 

This could be useful processing name data that come in the form: last,first or city,state. 

In [ ]:
# The movie genres are separated with the '|' character. 
# Remember, DataFrames can have columns of lists...

movies['genre_split'] = movies['genres'].str.split('|')
movies.head(10)

### str.join ( )
Put strings together. Separate the pieces with a delimiter of your choosing. 

In [ ]:
movies['with_colons'] = movies['genre_split'].str.join('::')
movies.head()

## Practice: What era had the best movies?
Take a few minutes and try the following. Feel free to chat with those around you if you get stuck.  

0. Reload 'movies.csv' into a DataFrame named 'movies_prac'. Work with 'movies_prac' for these exercises.
1. Extract the four-digit year from the titles and put them into a new column named 'year'.  

Notice that the year, including the parentheses is always the last 6 digits of the title. You might try `str.slice()` and work with negative indexes to count from the end of 'title'. 
2. There are 12 movies that do not have a year. Assign -1 to those. You might try the `str.isdigit()` method to see if the year you extracted in step 1. is numeric. 

3. Drop the movies without years.
4. Convert the year data to int (they should be strings). You might try `.astype()`. 
5. 'Bin' the years into decades (1899-1910; 1910-1920; 1920-1930;...2010-2020). Create a column called 'decade' to store the binned values.

6. Load 'ratings.csv' from the MovieLens data (name the DataFrame: `rate`) and merge it to `movies_prac` to create a new DataFrame named `movies_2`. Merge on 'movieId'. What kind of merge should you use (inner, outer, left, right)? We want to match every rating in `rate` with the information in `movies_prac`.

7. Run the following code. What does it do? We will learn more about `.groupby()` in the next class. 
```python
means = movies_2.groupby('decade')['rating'].mean()
```

According to Wikipedia (I know nothing about movies) the [Golden Age of Hollywood](https://en.wikipedia.org/wiki/Cinema_of_the_United_States#Classical_Hollywood_cinema_and_the_Golden_Age_of_Hollywood_(1917%E2%80%931960)) started in the late 1920s and ended in the 1960s. 


8. Create a horizontal bar chart with the average movie ratings. The y-axis labels should be the decades and the bars should represent the average movie rating. The index of `means` is an interval object. To assign it to the y-axis labels, convert it to a string 
```python
ax.barh(means.index.astype(str), means)
```